
# **Cuaderno de Proyecto — Ciencia de Datos con YouTube**
**Curso:** SINT-200  
**Profesor:** Dr. Tomás de Camino Beck  
**Estudiante(s):** _Bernal Rojas Villalobos_  
**Fecha de entrega:** 21 de Octubre

---

## Instrucciones Generales

Reto: Exportar datos de tu actividad YouTube (Google Takeout), construir una matriz usuario-contenido con señales (vistas, likes, tiempo, etc.), hacer EDA de sesgos/“burbujas”, y entrenar dos recomendadores (colaborativo vs. basado en contenido). Comparar métricas (cosas como precision@k, recall@k, cobertura) y explicar errores.  



Este cuaderno sirve como **especificación y entregable** del proyecto. Debes completar cada sección marcada con **TODO** y dejar celdas de código **ejecutables** y **reproducibles**. El reto tiene dos proyectos:

1. **Proyecto 1 — Tu Huella YouTube: Recomendador y Análisis de Burbuja Algorítmica.**  
2. **Proyecto 2 — Detección de “Doomscrolling”: Predicción de sesiones extendidas.**

### Ética y Privacidad de Datos
- Puedes **anonimizar** tu información de YouTube (IDs, títulos, canales, tiempos) antes de subirla aquí.  
- Alternativamente, puedes usar datos de otra persona **con su consentimiento informado** y **anonimizados**.  
- No incluyas PII (información personal identificable) ni material sensible.  
- Incluye un **Anexo de Privacidad** explicando qué datos usaste, cómo los obtuviste y cómo los protegiste.

### Entregables
- Este **cuaderno de Colab** completo y ejecutable.  
- Carpeta `data/` con **muestras** de los datos (o datos sintéticos/anonimizados).  
- **Diccionario de datos** (descripción de campos, tipos, unidades, supuestos).  
- **Resultados y visualizaciones** dentro del notebook.  
- **Conclusiones** + **Recomendaciones** (acciones sugeridas) + **Limitaciones** + **Trabajo futuro**.
- Repositorio con estructura mínima:  
  ```
  README.md
  data/        # muestras o datos anonimizados
  notebooks/   # este cuaderno
  src/         # funciones reutilizables
  reports/     # figuras / tablas clave
  ```

### Rúbrica (100 pts)
- **Charter/Problema y utilidad (10 pts)**: objetivos claros, hipótesis, valor para el usuario.  
- **Adquisición y calidad de datos (10 pts)**: trazabilidad, permisos, limpieza básica.  
- **EDA y visualizaciones (20 pts)**: distribución, outliers, correlaciones, sesgos/segmentos.  
- **Baselines y metodología (10 pts)**: definición de referencia simple y por qué.  
- **Modelado (20 pts)**: al menos **2 enfoques** comparados, justificación.  
- **Evaluación (15 pts)**: métricas adecuadas, validación (temporal cuando aplique), error analysis.  
- **Reproducibilidad (5 pts)**: semillas, funciones, estructura clara.  
- **Conclusiones & ética (10 pts)**: hallazgos accionables y reflexiones de privacidad/sesgo.

---



## 0. Preparación del entorno (ejecutar una vez)


In [1]:
# Clonar tu repositorio
!git clone https://github.com/brojas7/AnaliticaHistorialYoutube.git

# Ir al directorio del proyecto
%cd AnaliticaHistorialYoutube

# TODO: Ajusta versiones si lo necesitas. Evita dependencias innecesarias.
!pip -q install pandas numpy matplotlib scikit-learn textblob python-dateutil tqdm dateparser google-generativeai

Cloning into 'AnaliticaHistorialYoutube'...
remote: Enumerating objects: 76, done.
remote: Counting objects: 100% (76/76), done.
remote: Compressing objects: 100% (57/57), done.
remote: Total 76 (delta 20), reused 22 (delta 4), pack-reused 0 (from 0)
Receiving objects: 100% (76/76), 4.78 MiB | 10.45 MiB/s, done.
Resolving deltas: 100% (20/20), done.
/content/AnaliticaHistorialYoutube
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.5/315.5 kB 7.2 MB/s eta 0:00:00


In [11]:
# Imports base y configuración
import os, json, math, random, itertools, collections, gzip, re, string, time, zipfile, io
from datetime import datetime, timedelta
from dateutil import parser as dateparser
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, TimeSeriesSplit
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, roc_auc_score,
    confusion_matrix, mean_absolute_error, mean_squared_error
)

import sys
sys.path.append('/content/AnaliticaHistorialYoutube/src')

from youtube_utils import load_watch_history, anonymize_df, sessionize

# Reproducibilidad
SEED = 42
random.seed(SEED)
np.random.seed(SEED)

pd.set_option("display.max_columns", 120)
pd.set_option("display.max_rows", 200)

print("Entorno listo. Versión de pandas:", pd.__version__)


Entorno listo. Versión de pandas: 2.2.2



## 1. Anexo de Privacidad y Origen de Datos (obligatorio)
**TODO:** Explicar:
- Fuente de datos (Google Takeout, exportación manual, datos de tercero con consentimiento, etc.).  
- Estrategia de **anonimización** (por ejemplo: hashing de IDs/URLs, truncado de timestamps, agrupación por hora/día).  
- Contenido eliminado o agregado (p. ej., datos sintéticos para completar campos).  
- Limitaciones y riesgos residuales.



## 2. Selección de Proyecto
**Marca con una X**

- [ ] **Proyecto 1 — Recomendador YouTube & Burbuja Algorítmica**  
- [ ] **Proyecto 2 — Detección de Doomscrolling (clasificación temporal)**



## 3. Utilidades comunes para YouTube (ingesta y parsing)

Para **Proyecto 1** y **Proyecto 2** puedes usar datos de **Google Takeout**:  
- `watch-history.json` (o `watch-history.html` en exportaciones antiguas)  
- `search-history.json` (opcional)  
- `likes.csv` / `subscriptions.csv` (según disponibilidad)

> **Nota:** Los formatos de Takeout pueden cambiar con el tiempo. Ajusta el parser según tu exportación.



# **Limpieza, Enriquecimiento y Transformación del dataset**


In [ ]:
# Ejemplo de uso
df_search = load_watch_history('data/historial-de-búsqueda.json')
df_watch = load_watch_history('data/historial-de-reproducciones.json')
df_suscipciones = pd.read_csv('data/suscripciones.csv')

#df = anonymize_df(df)
#df = sessionize(df)
print(df_search.shape)
print(df_watch.shape)
print("Eventos:", len(df_watch), "Rango:", df_watch['timestamp'].min(), "->", df_watch['timestamp'].max())

(12090, 6)
(36842, 6)
Eventos: 36842 Rango: 2018-02-22 02:05:35.427000+00:00 -> 2025-10-18 01:02:57.606000+00:00


In [ ]:
# Copia base
df_watch_clean = df_watch.copy()

# Limpieza del título (remover "Has visto")
df_watch_clean['video_title'] = df_watch_clean['title'].str.replace(r'^Has visto\s+', '', regex=True)

# Convertir timestamps
df_watch_clean['watched_at'] = pd.to_datetime(df_watch_clean['timestamp'], utc=True)

# Variables derivadas
df_watch_clean['weekday'] = df_watch_clean['watched_at'].dt.day_name()
df_watch_clean['hour'] = df_watch_clean['watched_at'].dt.hour
df_watch_clean['hour_group'] = pd.cut(df_watch_clean['hour'],
                                      bins=[-1,6,12,18,24],
                                      labels=['madrugada','mañana','tarde','noche'])

In [ ]:
df_suscripciones_clean = df_suscipciones.rename(columns={
    "Título del canal": "channel_title",
    "URL del canal": "channel_url",
    "ID del canal": "channel_id"
})

In [ ]:
df_search_clean = df_search.copy()
df_search_clean['timestamp'] = pd.to_datetime(df_search_clean['timestamp'], utc=True)
df_search_clean['search_terms'] = df_search_clean['title'].str.replace('Buscaste', '').str.strip()

Enlace entre vistas y suscripciones

In [ ]:
# Renombramos la columna 'channel' en el historial para igualarla con suscripciones
df_watch_clean = df_watch_clean.rename(columns={'channel': 'channel_title'})

df_main = df_watch_clean.merge(
    df_suscripciones_clean[['channel_title', 'channel_id']],
    on='channel_title',
    how='left',
    indicator=True
)

df_main['is_subscribed'] = (df_main['_merge'] == 'both').astype(int)
df_main.drop(columns=['_merge'], inplace=True)

3️Señales de interacción (para matriz usuario–contenido)

In [ ]:
df_main['interaction_score'] = 1.0
df_main.loc[df_main['is_subscribed'] == 1, 'interaction_score'] += 0.5

Valores nulos en canal
son anuncios i videos borrados

In [ ]:
df_main.shape

(36842, 14)

In [ ]:
df_main[df_main['channel_title'].isna()][['video_title', 'url']].head(10)


,video_title,url
3,https://www.youtube.com/watch?v=l8dasJTQdw8,https://www.youtube.com/watch?v=l8dasJTQdw8
9,https://www.youtube.com/watch?v=Qh8vqGj_Dbg,https://www.youtube.com/watch?v=Qh8vqGj_Dbg
16,https://www.youtube.com/watch?v=vxDjs9wQrhw,https://www.youtube.com/watch?v=vxDjs9wQrhw
21,https://www.youtube.com/watch?v=09wmbNmZsbc,https://www.youtube.com/watch?v=09wmbNmZsbc
33,https://www.youtube.com/watch?v=or6p6659Jrg,https://www.youtube.com/watch?v=or6p6659Jrg
75,https://www.youtube.com/watch?v=vxDjs9wQrhw,https://www.youtube.com/watch?v=vxDjs9wQrhw
77,https://www.youtube.com/watch?v=UZwXbecjg1Y,https://www.youtube.com/watch?v=UZwXbecjg1Y
81,https://www.youtube.com/watch?v=09wmbNmZsbc,https://www.youtube.com/watch?v=09wmbNmZsbc
97,https://www.youtube.com/watch?v=EM9OG65ccVI,https://www.youtube.com/watch?v=EM9OG65ccVI
185,https://www.youtube.com/watch?v=educpLKQbEg,https://www.youtube.com/watch?v=educpLKQbEg


In [ ]:
mask_valid = df_main['channel_title'].notna() & ~df_main['video_title'].str.contains("anuncio", case=False, na=False)
df_main_clean = df_main[mask_valid].copy()

In [ ]:
df_ads = df_main[~mask_valid]
df_ads.to_csv("data/df_ads_removed.csv", index=False)

ANON


# **Proyecto 1 — Tu Huella YouTube: Recomendador & Burbuja Algorítmica**

### Objetivo
1) Construir **dos recomendadores** con tus datos de visualización:  
   - **Baseline de popularidad** (o popularidad por canal/categoría).  
   - **Modelo basado en contenido** (TF‑IDF/embeddings por título/canal) **o** **colaborativo** (si tienes interacciones de múltiples usuarios/fuentes).  
2) Medir **Precision@k, Recall@k y Coverage** (y *diversidad*) en un esquema **offline**.  
3) Analizar posibles **sesgos o “burbujas”** (temas/canales dominantes por hora, día, duración).  

### Requisitos mínimos
- **EDA**: distribución de vistas por canal, hora del día, día de semana, duración de sesiones, *top‑k* temas.  
- **Ingeniería de features** (ej.: tokenización títulos, lematización opcional, normalización de canales).  
- **Comparación de al menos 2 enfoques** de recomendación.  
- **Evaluación offline** con *train/test split temporal*.  
- **Análisis de errores** y discusión de sesgos/limitaciones.

---

## 4. Charter del Proyecto 1 (llenar)
**TODO:** Define el propósito, preguntas clave y utilidad (qué decisiones permitirán tus hallazgos).



## 5. Carga de datos (Proyecto 1)
**TODO:** Sube tu `watch-history.json` (anonimizado si aplica) a `data/` y cárgalo.


In [4]:
df_main = pd.read_csv("data/df_enrich_enriquecido.csv")
print(df_main.shape)
df_main.head()

(31217, 18)


,timestamp,title,channel_title,channel_id_x,video_id,url,video_title,watched_at,weekday,hour,hour_group,channel_id_y,is_subscribed,interaction_score,category,subtopic,format,keywords
0,2018-02-22 02:05:35.427000+00:00,Has visto La canción más hermosa en piano fáci...,Paula Yessenia Barragan Izquierdo,NaN,ro1rC9dL5EQ,https://www.youtube.com/watch?v=ro1rC9dL5EQ,La canción más hermosa en piano fácil de aprender,2018-02-22 02:05:35.427000+00:00,Thursday,2,madrugada,NaN,0,1.0,Tutorial,Music Lesson,Tutorial,"['piano', 'easy to learn', 'music lesson']"
1,2018-02-22 03:49:54.096000+00:00,Has visto Ed Sheeran - Perfect - EASY Piano Tu...,Peter PlutaX,NaN,p1WCR7vNcIw,https://www.youtube.com/watch?v=p1WCR7vNcIw,Ed Sheeran - Perfect - EASY Piano Tutorial by ...,2018-02-22 03:49:54.096000+00:00,Thursday,3,madrugada,NaN,0,1.0,Tutorial,Music Lesson,Tutorial,"['Ed Sheeran', 'Perfect', 'piano tutorial', 'e..."
2,2018-02-22 05:27:06.367000+00:00,Has visto How I proposed: fairytale story with...,bigchewypretzels,NaN,Srw9gXDa24g,https://www.youtube.com/watch?v=Srw9gXDa24g,How I proposed: fairytale story with puzzles a...,2018-02-22 05:27:06.367000+00:00,Thursday,5,madrugada,NaN,0,1.0,Entertainment,Relationship/Life Event,Vlog/Storytelling,"['proposal', 'love story', 'fairytale', 'relat..."
3,2018-02-22 06:26:38.438000+00:00,Has visto TOP 5 BROMAS - Bromas para hacer a t...,BROMAS Y MÁS TVOAQUI,NaN,Q4UilByqMTc,https://www.youtube.com/watch?v=Q4UilByqMTc,"TOP 5 BROMAS - Bromas para hacer a tus amigos,...",2018-02-22 06:26:38.438000+00:00,Thursday,6,madrugada,NaN,0,1.0,Entertainment,Comedy/Prank,Prank Compilation,"['pranks', 'friends', ""Valentine's Day"", 'come..."
4,2018-02-22 14:20:32.963000+00:00,Has visto Morat - Yo Más Te Adoro,MoratVEVO,NaN,pqJBXjzBr_U,https://www.youtube.com/watch?v=pqJBXjzBr_U,Morat - Yo Más Te Adoro,2018-02-22 14:20:32.963000+00:00,Thursday,14,tarde,NaN,0,1.0,Music,Music Performance,Official Music Video,"['Morat', 'Yo Más Te Adoro', 'Latin Pop', 'Off..."


In [8]:
df_main.subtopic.value_counts()

,count
subtopic,
Latin Pop,1399
Reggaeton,971
Bachata,570
Pop,464
Latin Rock,352
...,...
Home Safety,1
"Latin Pop, Singer-Songwriter",1
"Traditional Mexican, Folk",1


In [24]:
import hashlib
import pandas as pd
import numpy as np

def hash_value(x):
    """Aplica hash SHA-256 truncado (12 chars) a cualquier valor."""
    if pd.isna(x):
        return np.nan
    return hashlib.sha256(str(x).encode()).hexdigest()[:12]

# Copia del dataframe original
df_hashed = df_main.copy()

# Aplica el hash a todo el dataframe
df_hashed = df_hashed.applymap(hash_value)

# Verifica resultado
df_hashed.head()

/tmp/ipython-input-359848926.py:15: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_hashed = df_hashed.applymap(hash_value)


,timestamp,title,channel_title,channel_id_x,video_id,url,video_title,watched_at,weekday,hour,hour_group,channel_id_y,is_subscribed,interaction_score,category,subtopic,format,keywords
0,43b624cafb0c,b5c2fe6c821d,561cc3b6b7fb,NaN,d8b7eb145254,5b328611dfd1,7f78ca26e123,43b624cafb0c,fc2662062ffd,d4735e3a265e,c420fdb9ac10,NaN,5feceb66ffc8,d0ff5974b6aa,c26982b1425d,1ecbd27cfc92,c26982b1425d,1dc189d46c1d
1,ac49439f1641,9267aa5a5ce2,bc5af354f824,NaN,7e655fd0d398,2aed642c7f9d,11eaa6ac0ea2,ac49439f1641,fc2662062ffd,4e07408562be,c420fdb9ac10,NaN,5feceb66ffc8,d0ff5974b6aa,c26982b1425d,1ecbd27cfc92,c26982b1425d,405ab863bc17
2,aa80c41676a7,5035d49868e3,07254e8fb92b,NaN,585412c8f79b,3688faaf95fb,d954a349049f,aa80c41676a7,fc2662062ffd,ef2d127de37b,c420fdb9ac10,NaN,5feceb66ffc8,d0ff5974b6aa,ceaa553e838f,4fae335077e6,200fafff5b00,c75c87529646
3,d6273362f33c,bc92ec09f80c,b03d217c9070,NaN,ad81fdd792ac,0d89a0111957,77d86ffa77c6,d6273362f33c,fc2662062ffd,e7f6c011776e,c420fdb9ac10,NaN,5feceb66ffc8,d0ff5974b6aa,ceaa553e838f,d63b6b556ce7,ff14d2076ecc,3edeeb3b1efd
4,52c804da8e4d,f0f24a0474ac,0f66b5b56794,NaN,4e7403885505,018b4f680689,c13793508e6c,52c804da8e4d,fc2662062ffd,8527a891e224,dfc02909a308,NaN,5feceb66ffc8,d0ff5974b6aa,6eb00b4b2614,9423a442e22b,754dc3f10982,f642e0d4921f


In [30]:
df_main.hour_group.value_counts()

,count
hour_group,
tarde,11342
noche,9642
madrugada,8504
mañana,1729



## 6. EDA (Proyecto 1)
**TODO:** Explora sesgos por canal/tema/horario. Muestra tablas y gráficos clave.


In [34]:
df_main.head()

,timestamp,title,channel_title,channel_id_x,video_id,url,video_title,watched_at,weekday,hour,hour_group,channel_id_y,is_subscribed,interaction_score,category,subtopic,format,keywords
0,2018-02-22 02:05:35.427000+00:00,Has visto La canción más hermosa en piano fáci...,Paula Yessenia Barragan Izquierdo,NaN,ro1rC9dL5EQ,https://www.youtube.com/watch?v=ro1rC9dL5EQ,La canción más hermosa en piano fácil de aprender,2018-02-22 02:05:35.427000+00:00,Thursday,2,madrugada,NaN,0,1.0,Tutorial,Music Lesson,Tutorial,"['piano', 'easy to learn', 'music lesson']"
1,2018-02-22 03:49:54.096000+00:00,Has visto Ed Sheeran - Perfect - EASY Piano Tu...,Peter PlutaX,NaN,p1WCR7vNcIw,https://www.youtube.com/watch?v=p1WCR7vNcIw,Ed Sheeran - Perfect - EASY Piano Tutorial by ...,2018-02-22 03:49:54.096000+00:00,Thursday,3,madrugada,NaN,0,1.0,Tutorial,Music Lesson,Tutorial,"['Ed Sheeran', 'Perfect', 'piano tutorial', 'e..."
2,2018-02-22 05:27:06.367000+00:00,Has visto How I proposed: fairytale story with...,bigchewypretzels,NaN,Srw9gXDa24g,https://www.youtube.com/watch?v=Srw9gXDa24g,How I proposed: fairytale story with puzzles a...,2018-02-22 05:27:06.367000+00:00,Thursday,5,madrugada,NaN,0,1.0,Entertainment,Relationship/Life Event,Vlog/Storytelling,"['proposal', 'love story', 'fairytale', 'relat..."
3,2018-02-22 06:26:38.438000+00:00,Has visto TOP 5 BROMAS - Bromas para hacer a t...,BROMAS Y MÁS TVOAQUI,NaN,Q4UilByqMTc,https://www.youtube.com/watch?v=Q4UilByqMTc,"TOP 5 BROMAS - Bromas para hacer a tus amigos,...",2018-02-22 06:26:38.438000+00:00,Thursday,6,madrugada,NaN,0,1.0,Entertainment,Comedy/Prank,Prank Compilation,"['pranks', 'friends', ""Valentine's Day"", 'come..."
4,2018-02-22 14:20:32.963000+00:00,Has visto Morat - Yo Más Te Adoro,MoratVEVO,NaN,pqJBXjzBr_U,https://www.youtube.com/watch?v=pqJBXjzBr_U,Morat - Yo Más Te Adoro,2018-02-22 14:20:32.963000+00:00,Thursday,14,tarde,NaN,0,1.0,Music,Music Performance,Official Music Video,"['Morat', 'Yo Más Te Adoro', 'Latin Pop', 'Off..."


In [ ]:
!pip install gensim -q
import gensim.downloader as api
model = api.load("word2vec-google-news-300")

[========------------------------------------------] 17.2% 286.7/1662.8MB downloaded

In [36]:
 Función para obtener el embedding promedio de una lista de palabras
def get_embedding(tokens, model):
    valid = [t for t in tokens if t in model]
    if not valid:
        return np.zeros(model.vector_size)
    return np.mean([model[t] for t in valid], axis=0)

# Combinar campos textuales
cols_text = ["category", "subtopic", "format", "keywords", "video_title", "channel_title"]

def combine_text(row):
    tokens = []
    for col in cols_text:
        val = row[col]
        if isinstance(val, list):
            tokens.extend(val)
        elif isinstance(val, str):
            tokens.extend(val.lower().split())
    return tokens

df_embed = df_main.copy()
df_embed["tokens_all"] = df_embed.apply(combine_text, axis=1)

# Obtener el embedding promedio por fila
df_embed["embedding"] = df_embed["tokens_all"].apply(lambda x: get_embedding(x, model))

# Convertir embeddings en columnas
embeddings = np.vstack(df_embed["embedding"].values)
df_embeddings = pd.DataFrame(embeddings, columns=[f"emb_{i}" for i in range(model.vector_size)])

# Concatenar con el dataframe original si quieres
df_final = pd.concat([df_main.reset_index(drop=True), df_embeddings], axis=1)

df_final.head()


## 7. Partición temporal y definición de tareas (Proyecto 1)
**TODO:** Define ventana de entrenamiento y de evaluación para recomendación **offline**.


In [ ]:

# Split temporal: por ejemplo, último 20% del tiempo como test
cut_ts = df_watch['timestamp'].quantile(0.8)
train = df_watch[df_watch['timestamp'] <= cut_ts].copy()
test  = df_watch[df_watch['timestamp'] >  cut_ts].copy()

print("train:", train['timestamp'].min(), "->", train['timestamp'].max(), "n=", len(train))
print("test :", test['timestamp'].min(),  "->", test['timestamp'].max(),  "n=", len(test))



## 8. Baseline de popularidad (Proyecto 1)
Genera recomendaciones **sin personalización** como referencia.


In [ ]:

# Top-N por popularidad (baseline)
K = 10  # tamaño de recomendación
top_items = train['video_id'].value_counts().head(100).index.tolist()

def recommend_popularity(k=K):
    return top_items[:k]

# Conjunto de ítems verdaderos en test (lo visto en test)
true_items = set(test['video_id'].dropna().unique().tolist())

def precision_at_k(recommended, true_set):
    if len(recommended) == 0: return 0.0
    hit = sum(1 for x in recommended if x in true_set)
    return hit / len(recommended)

def recall_at_k(recommended, true_set):
    if len(true_set) == 0: return 0.0
    hit = sum(1 for x in recommended if x in true_set)
    return hit / len(true_set)

# Eval baseline
rec = recommend_popularity(K)
p = precision_at_k(rec, true_items)
r = recall_at_k(rec, true_items)
coverage = len(set(top_items)) / max(1, df_watch['video_id'].nunique())

print(f"Baseline Popularidad -> P@{K}={p:.3f}  R@{K}={r:.3f}  Cobertura={coverage:.3f}")


## 9. Recomendador basado en contenido **(ejemplo TF-IDF por título/canal)**
**TODO:** Implementa TF‑IDF (o embeddings) y calcula similitud contenido‑a‑contenido para recomendar.


In [ ]:

# EJEMPLO ESQUELETO (completa con TF-IDF real si lo deseas)
# Aquí usamos una heurística mínima por canal/tokens de título para ilustrar el flujo.

from collections import Counter

def tokenize_title(s):
    if pd.isna(s): return []
    s = s.lower()
    s = re.sub(r"[^a-z0-9áéíóúüñ\s]", " ", s)
    tok = [t for t in s.split() if len(t) > 2]
    return tok

train = train.copy()
train['tokens'] = train['title'].apply(tokenize_title)

# "Perfil" de intereses por tokens (muy básico)
profile = Counter(itertools.chain.from_iterable(train['tokens'].tolist()))
top_terms = [t for t, _ in profile.most_common(50)]

def recommend_content_based(k=10):
    # Recomienda ítems del set de entrenamiento por coincidencia con términos del perfil
    scores = []
    for vid, grp in train.groupby('video_id'):
        toks = list(itertools.chain.from_iterable(grp['tokens'].tolist()))
        score = sum(1 for t in toks if t in top_terms)
        scores.append((vid, score))
    scores.sort(key=lambda x: x[1], reverse=True)
    return [vid for vid, s in scores[:k]]

rec_cb = recommend_content_based(K)
p_cb = precision_at_k(rec_cb, true_items)
r_cb = recall_at_k(rec_cb, true_items)
cov_cb = len(set(train['video_id'])) / max(1, df_watch['video_id'].nunique())
print(f"Contenido (heurístico) -> P@{K}={p_cb:.3f}  R@{K}={r_cb:.3f}  Cobertura={cov_cb:.3f}")

# TODO: Sustituir por TF-IDF/embeddings reales y mejorar evaluación (por usuario/ventanas).



## 10. Análisis de burbuja/sesgo (Proyecto 1)
**TODO:** Mide concentración por canal/tema, horarios de consumo, diversidad de recomendaciones.
Propón **intervenciones** para aumentar diversidad sin perder pertinencia.



## 11. Conclusiones y trabajo futuro (Proyecto 1)
**TODO:** Resume hallazgos, limitaciones y siguientes pasos.



# **Proyecto 2 — Detección de “Doomscrolling” (Clasificación Temporal)**

### Objetivo
Predecir si una **sesión** de consumo (YouTube u otra plataforma) terminará en **scroll extendido** o **consumo prolongado** (> X minutos o > Y videos).

### Requisitos mínimos
- **Definición de etiqueta** (doomscroll = 1 si supera umbral de tiempo o conteo).  
- **Sessionization** con ventana de inactividad (p. ej., 30 minutos).  
- **Features temporales** (hora/día, ritmos, gaps), contextuales (tema/canal) y de **acumulación**.  
- **Validación temporal** (e.g., TimeSeriesSplit) para evitar *leakage*.  
- **Comparación de al menos 2 clasificadores** (baseline + modelo).  
- **Métricas**: precisión, recall, F1, ROC‑AUC, matriz de confusión por segmento.  
- **Error analysis** e importancia de variables.

---

## 12. Charter del Proyecto 2 (llenar)
**TODO:** Propósito, hipótesis y utilidad (p. ej., alertas o cortes saludables de uso).



## 13. Carga de datos (Proyecto 2)
**TODO:** Reutiliza `watch-history.json` u otra fuente y genera sesiones.


In [ ]:

# Ejemplo de carga y sessionization
PATH_WATCH = "data/watch-history.json"  # TODO: cambia si es necesario
df_watch = load_watch_history(PATH_WATCH)

# Crea sesiones (ajusta gap si aplica)
df_sess = sessionize(df_watch, gap_minutes=30)

# Etiqueta doomscrolling: duración > umbral (estimación proxy por número de eventos)
UMBRAL_VIDEOS = 8  # TODO: ajusta criterio (o usa tiempo real si lo tienes)
sess_counts = df_sess.groupby('session_id').size().rename('n_events')
df_labels = sess_counts.to_frame().assign(doom=lambda x: (x['n_events'] >= UMBRAL_VIDEOS).astype(int)).reset_index()

print(df_labels['doom'].value_counts())
df_labels.head()



## 14. Features y partición temporal (Proyecto 2)
**TODO:** Crear variables por sesión (hora de inicio, día de semana, gaps promedio, temas/canales dominantes, ritmo).  
Usa **TimeSeriesSplit** o una corte temporal para evaluación.


In [ ]:

# Construcción simple de features por sesión (extiende según tus datos)
def build_session_features(df_watch, df_sess):
    # Hora de inicio de la sesión, día de semana, tamaño de sesión, ritmo aproximado
    start_ts = df_sess.groupby('session_id')['timestamp'].min().rename('start_ts')
    end_ts   = df_sess.groupby('session_id')['timestamp'].max().rename('end_ts')
    n_events = df_sess.groupby('session_id').size().rename('n_events')

    base = pd.concat([start_ts, end_ts, n_events], axis=1).reset_index()
    base['duration_min'] = (base['end_ts'] - base['start_ts']).dt.total_seconds() / 60.0
    base['hour'] = base['start_ts'].dt.hour
    base['dow'] = base['start_ts'].dt.dayofweek  # 0=Lunes
    base['events_per_min'] = base['n_events'] / base['duration_min'].replace(0, np.nan)

    # (Opcional) top canal por sesión
    top_channel = (
        df_sess.groupby(['session_id','channel']).size()
        .reset_index(name='cnt')
        .sort_values(['session_id','cnt'], ascending=[True, False])
        .drop_duplicates('session_id')
        .set_index('session_id')['channel']
        .rename('top_channel')
    )
    base = base.merge(top_channel, left_on='session_id', right_index=True, how='left')

    # Codificación simple de canal principal (dummy)
    if 'top_channel' in base.columns:
        dummies = pd.get_dummies(base['top_channel'], prefix='ch', dummy_na=True)
        base = pd.concat([base.drop(columns=['top_channel']), dummies], axis=1)

    return base

df_feat = build_session_features(df_watch, df_sess)
df_all = df_feat.merge(df_labels, on='session_id', how='inner')

# Partición temporal (último 20% del tiempo como test)
cut_ts = df_all['start_ts'].quantile(0.8)
train = df_all[df_all['start_ts'] <= cut_ts].copy()
test  = df_all[df_all['start_ts'] >  cut_ts].copy()

y_train = train['doom'].values
y_test  = test['doom'].values

X_train = train.drop(columns=['doom','start_ts','end_ts'])
X_test  = test.drop(columns=['doom','start_ts','end_ts'])

print("Train/Test shapes:", X_train.shape, X_test.shape)



## 15. Baseline y Modelo(s) (Proyecto 2)
**TODO:** Compara un baseline (mayoría/clasificador trivial) con al menos **un modelo** (p. ej., Árboles/GBM/RegLog).


In [ ]:

# Baseline: clase mayoritaria
from collections import Counter
majority = Counter(y_train).most_common(1)[0][0]
y_pred_base = np.full_like(y_test, fill_value=majority)
print("Baseline (mayoría) -> acc=%.3f  prec=%.3f  rec=%.3f  f1=%.3f" % (
    accuracy_score(y_test, y_pred_base),
    precision_score(y_test, y_pred_base, zero_division=0),
    recall_score(y_test, y_pred_base, zero_division=0),
    f1_score(y_test, y_pred_base, zero_division=0)
))

# Modelo ejemplo: Regresión Logística (puedes sustituir por otro)
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(max_iter=1000, random_state=SEED)
clf.fit(X_train.fillna(0), y_train)
y_proba = clf.predict_proba(X_test.fillna(0))[:,1]
y_pred = (y_proba >= 0.5).astype(int)

print("LogReg -> acc=%.3f  prec=%.3f  rec=%.3f  f1=%.3f  roc=%.3f" % (
    accuracy_score(y_test, y_pred),
    precision_score(y_test, y_pred, zero_division=0),
    recall_score(y_test, y_pred, zero_division=0),
    f1_score(y_test, y_pred, zero_division=0),
    roc_auc_score(y_test, y_proba) if len(np.unique(y_test))>1 else float('nan')
))



## 16. Análisis de errores e importancia de variables (Proyecto 2)
**TODO:** Muestra matriz de confusión, curvas y comenta falsos positivos/negativos por segmento.


In [ ]:

# Matriz de confusión (numérica)
cm = confusion_matrix(y_test, y_pred)
cm



## 17. Conclusiones y trabajo futuro (Proyecto 2)
**TODO:** Resume hallazgos, utilidad práctica (alertas, límites de tiempo, recomendaciones) y próximos pasos.



---

## 18. Limitaciones generales y reflexiones éticas
**TODO:** Reflexiona sobre sesgos, representatividad de datos personales, efectos de recomendación, privacidad y seguridad.

## 19. Checklist final
- [ ] El cuaderno **corre desde cero** (Runtime -> Restart & Run All).  
- [ ] Incluye **diccionario de datos** y **anexo de privacidad**.  
- [ ] Tiene **baselines** y **≥2 modelos** comparados (según el proyecto).  
- [ ] Reporta **métricas** adecuadas y **análisis de errores**.  
- [ ] Conclusiones accionables y **trabajo futuro**.

